# regression ultimate code

In [1]:
import numpy as np
import argparse

import os
import math
from math import sin

import json

import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')
matplotlib.rc( 'savefig', facecolor = 'white' )
from matplotlib import pyplot
# %matplotlib inline

import sys
sys.path.append('../') # add this line so Data and data are visible in this file
sys.path.append('../../') # add this line so Data and data are visible in this file

In [2]:
parser = argparse.ArgumentParser(description='EE_estimate')
parser.add_argument('--input_folder', metavar='input_folder', help='input_folder',
                    default='../')
parser.add_argument('--output_folder', metavar='output_folder', help='output_folder',
                    default='../')


args = parser.parse_args(['--input_folder', '../../data/stage3/win60_overlap90/', 
                          '--output_folder', '../../data/collaboration/ML_regression/',
                         ])
# args = parser.parse_args()
print(args)

Namespace(input_folder='../../data/stage3/win60_overlap90/', output_folder='../../data/collaboration/ML_regression/')


In [3]:
inputdir = args.input_folder
outputdir = args.output_folder

if not os.path.exists(outputdir):
    os.makedirs(outputdir)

# load stage3_dict, which is a disctionary that stores key information about the dataset

In [4]:
def data_loader(name, inputdir):
    """ usage: data = data_loader('data', outputdir)"""
    inputdir_data = os.path.join(inputdir, name+'.npz')
    data = np.load(inputdir_data, allow_pickle=True)['data']
    return data

In [5]:
stage3_dict = data_loader('stage3_dict', inputdir).item()

list_feature = stage3_dict['list_feature']
list_meta = stage3_dict['list_meta']
list_output = stage3_dict['list_output']
unit_dict = stage3_dict['unit_dict']
tasks_dict = stage3_dict['tasks_dict']
tasks_dict_reversed = stage3_dict['tasks_dict_reversed']

# load the extracted features, label, and meta data (they should have the same number of rows)

In [7]:
feature_all = data_loader('feature', inputdir)
label_all = data_loader('label', inputdir)
meta_all = data_loader('meta', inputdir)

In [8]:
print(feature_all.shape, len(list_feature))
print(meta_all.shape,  len(list_meta))
print(label_all.shape, len(list_output))

(2524, 21) 21
(2524, 5) 5
(2524, 14) 14


# use only a subset of the subjects and tasks

In [9]:
np.unique(meta_all[:,0])

KEEP_subject_ids = np.asarray([104, 105, 106, 107, 108, 110, 111, 113, 114, 115, 116, 117, 118, 119, 120, 121, 212])
mask_subjects = np.in1d(meta_all[:,0], KEEP_subject_ids)

KEEP_task_ids = np.asarray([0, 1, 2, 3, 4, 5]) # all recovery tasks. Reference: `tasks_dict` 
mask_tasks = np.in1d(meta_all[:,1], KEEP_task_ids)


feature_all = feature_all[mask_subjects & mask_tasks, :]
label_all = label_all[mask_subjects & mask_tasks, :]
meta_all = meta_all[mask_subjects & mask_tasks, :]

print(feature_all.shape, len(list_feature))
print(meta_all.shape,  len(list_meta))
print(label_all.shape, len(list_output))

(2042, 21) 21
(2042, 5) 5
(2042, 14) 14


# please start with a subset of the features you are familiar with below
## do not hesistate to reach out to me if you have any questions!